# 5

In [2]:
import pandas as pd
import numpy as np
import wooldridge as woo
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import scipy.stats
import numpy.ma as ma
import patsy as pt
import statsmodels.stats.outliers_influence as smo
import statsmodels.api as sm
import linearmodels.iv as iv
from simple_colors import *
from RegscorePy import mallow
from sklearn.model_selection import KFold
import warnings 

## PART 1

### The data in $ injury $ was used to study the length of time (in weeks) that an injured worker receives workers’ compensation.

In [2]:
inj = woo.data('INJURY')
inj.head()

,durat,afchnge,highearn,male,married,hosp,indust,injtype,age,prewage,...,head,neck,upextr,trunk,lowback,lowextr,occdis,manuf,construc,highlpre
0,1.0,1,1,1.0,0.0,1,3.0,1,26.0,404.950012,...,1,0,0,0,0,0,0,0.0,0.0,6.003764
1,1.0,1,1,1.0,1.0,0,3.0,1,31.0,643.825012,...,1,0,0,0,0,0,0,0.0,0.0,6.467427
2,84.0,1,1,1.0,1.0,1,3.0,1,37.0,398.125000,...,1,0,0,0,0,0,0,0.0,0.0,5.986766
3,4.0,1,1,1.0,1.0,1,3.0,1,31.0,527.799988,...,1,0,0,0,0,0,0,0.0,0.0,6.268717
4,1.0,1,1,1.0,1.0,0,3.0,1,23.0,528.937500,...,1,0,0,0,0,0,0,0.0,0.0,6.270870


In [3]:
inj.describe()

,durat,afchnge,highearn,male,married,hosp,indust,injtype,age,prewage,...,head,neck,upextr,trunk,lowback,lowextr,occdis,manuf,construc,highlpre
count,7150.000000,7150.000000,7150.000000,7134.000000,6853.000000,7150.000000,7125.000000,7150.000000,7146.000000,7150.000000,...,7150.000000,7150.000000,7150.000000,7150.000000,7150.000000,7150.000000,7150.000000,7125.000000,7125.000000,7150.000000
mean,9.922203,0.473287,0.398881,0.780628,0.692252,0.262098,2.292491,4.450909,34.705849,329.728482,...,0.036364,0.017063,0.295245,0.114126,0.261818,0.231608,0.010769,0.280842,0.145825,2.484137
std,24.497542,0.499321,0.489702,0.413850,0.461596,0.439806,0.876774,1.516924,12.590252,182.798939,...,0.187206,0.129515,0.456185,0.317986,0.439655,0.421890,0.103222,0.449442,0.352955,3.053281
min,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,12.000000,81.780602,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,3.000000,24.000000,184.625900,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.000000,0.000000,0.000000,1.000000,1.000000,0.000000,3.000000,5.000000,32.000000,263.850006,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,8.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,6.000000,43.000000,449.673103,...,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,6.108521
max,182.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,8.000000,98.000000,1583.099976,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.367140


In [4]:
inj.isnull().sum()

durat         0
afchnge       0
highearn      0
male         16
married     297
hosp          0
indust       25
injtype       0
age           4
prewage       0
totmed        0
injdes        0
benefit       0
ky            0
mi            0
ldurat        0
afhigh        0
lprewage      0
lage          4
ltotmed       0
head          0
neck          0
upextr        0
trunk         0
lowback       0
lowextr       0
occdis        0
manuf        25
construc     25
highlpre      0
dtype: int64

### The control group is low-income workers, and the treatment group is high-income workers. Estimate and report the treatment effect on $ log(durat) $ and $ highearn $ as the dummy variable for high earners. Include controls for gender, marital status, age, and type of injury.

In [7]:
inj["highearn"]

0       1
1       1
2       1
3       1
4       1
       ..
7145    0
7146    0
7147    0
7148    0
7149    0
Name: highearn, Length: 7150, dtype: int64

In [8]:
durat_fitC = smf.ols('np.log(durat) ~ highearn + male + married + age + C(injtype)', inj).fit()
durat_fitC.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(durat)   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     22.61
Date:                Tue, 29 Nov 2022   Prob (F-statistic):           4.02e-46
Time:                        11:16:43   Log-Likelihood:                -11397.
No. Observations:                6844   AIC:                         2.282e+04
Df Residuals:                    6832   BIC:                         2.290e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.4625      0.098      4.705      0.000       0.270       0.655
C(injtype)[T.2]     0.7661      0.145      5.294      0.000       0.482       1.050
C(injtype)[T.3]     0.3825      0.086      4.468      0.000       0.215       0.550
C(injtype)[T.4]     0.6969      0.093      7.511      0.000       0.515       0.879
C(injtype)[T.5]     0.4759      0.086      5.519      0.000       0.307       0.645
C(injtype)[T.6]     0.4188      0.087      4.822      0.000       0.249       0.589
C(injtype)[T.7]     0.9108      0.175      5.219      0.000       0.569       1.253
C(injtype)[T.8]     0.5926      0.117      5.054      0.000       0.363       0.822
highearn            0.2470      0.036      6.931      0.000       0.177       0.317
male               -0.0341      0.041     -0.835      0.404      -0.114       0.046
married             0.0484      0.036      1.337      0.181      -0.023       0.119
age                 0.0089      0.001      6.813      0.000       0.006       0.012
==============================================================================
Omnibus:                       58.949   Durbin-Watson:                   1.918
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               92.702
Skew:                           0.039   Prob(JB):                     7.41e-21
Kurtosis:                       3.565   Cond. No.                         585.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
print("The coefficent for highearn is:", durat_fitC.params[8])

The coefficent for highearn is: 0.2470087729409308


The coefficient for $highearn$ is telling us that the treatment effect on the dependent variable is 24.7%; high-hearners increase the duration of the benefit by that amount.

### Test whether assignment to the treatment groups is random.

In [12]:
earn_fitC = smf.ols('highearn ~ male + married + age + C(injtype)', inj).fit()
earn_fitC.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               highearn   R-squared:                       0.209
Model:                            OLS   Adj. R-squared:                  0.208
Method:                 Least Squares   F-statistic:                     180.7
Date:                Tue, 29 Nov 2022   Prob (F-statistic):               0.00
Time:                        11:25:23   Log-Likelihood:                -4003.1
No. Observations:                6844   AIC:                             8028.
Df Residuals:                    6833   BIC:                             8103.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.2527      0.033     -7.605      0.000      -0.318      -0.188
C(injtype)[T.2]     0.1578      0.049      3.215      0.001       0.062       0.254
C(injtype)[T.3]    -0.0502      0.029     -1.726      0.084      -0.107       0.007
C(injtype)[T.4]    -0.0008      0.031     -0.026      0.979      -0.063       0.061
C(injtype)[T.5]     0.0094      0.029      0.320      0.749      -0.048       0.067
C(injtype)[T.6]     0.0081      0.029      0.275      0.783      -0.050       0.066
C(injtype)[T.7]     0.0656      0.059      1.107      0.269      -0.051       0.182
C(injtype)[T.8]     0.0026      0.040      0.065      0.948      -0.075       0.081
male                0.4066      0.013     31.403      0.000       0.381       0.432
married             0.2257      0.012     18.813      0.000       0.202       0.249
age                 0.0052      0.000     11.720      0.000       0.004       0.006
==============================================================================
Omnibus:                    10932.090   Durbin-Watson:                   0.331
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              471.515
Skew:                           0.137   Prob(JB):                    4.09e-103
Kurtosis:                       1.744   Cond. No.                         585.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

As it can be observed in the table result, the f p-value is small. Therefore it can be concluded that the assignment to the treatment group is not random.

### On July 15, 1980, Kentucky raised the cap on weekly earnings that were covered by workers’ compensation. An increase in the cap has no effect on the benefit for low-income workers, but it makes it less costly for a high-income worker to stay on workers’ compensation. Use difference-in-differences analysis to estimate the treatment effect. Let afchnge be the dummy variable for observations after the policy change.

In [12]:
duratdif_fit = smf.ols('np.log(durat) ~ afchnge + highearn + afchnge*highearn', inj).fit()
duratdif_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(durat)   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     38.34
Date:                Tue, 29 Nov 2022   Prob (F-statistic):           1.42e-24
Time:                        10:22:52   Log-Likelihood:                -12011.
No. Observations:                7150   AIC:                         2.403e+04
Df Residuals:                    7146   BIC:                         2.406e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            1.1993      0.027     44.241      0.000       1.146       1.252
afchnge              0.0236      0.040      0.595      0.552      -0.054       0.101
highearn             0.2152      0.043      4.963      0.000       0.130       0.300
afchnge:highearn     0.1883      0.063      2.999      0.003       0.065       0.311
==============================================================================
Omnibus:                       61.563   Durbin-Watson:                   1.899
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               91.572
Skew:                           0.078   Prob(JB):                     1.30e-20
Kurtosis:                       3.532   Cond. No.                         6.28
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
print("The treatment effect on duration of the benefit is:", duratdif_fit.params[3]*100, "%")

The treatment effect on duration of the benefit is: 18.83497798445467 %


The economic meaning is that the average duration of benefit for high earners increased by about 18.83% due to the change in salary cap. 

## PART 2

### The data in FERTIL2 include, for women in Botswana during 1988, information on number of children, years of education, age, and religious and economic status variables.

In [28]:
fertil = woo.data('FERTIL2')
fertil.head()

,mnthborn,yearborn,age,electric,radio,tv,bicycle,educ,ceb,agefbrth,...,heduc,agesq,urban,urb_educ,spirit,protest,catholic,frsthalf,educ0,evermarr
0,5,64,24,1.0,1.0,1.0,1.0,12,0,NaN,...,NaN,576,1,12,0,0,0,1,0,0
1,1,56,32,1.0,1.0,1.0,1.0,13,3,25.0,...,12.0,1024,1,13,0,0,0,1,0,1
2,7,58,30,1.0,0.0,0.0,0.0,5,1,27.0,...,7.0,900,1,5,1,0,0,0,0,1
3,11,45,42,1.0,0.0,1.0,0.0,4,3,17.0,...,11.0,1764,1,4,0,0,0,0,0,1
4,5,45,43,1.0,1.0,1.0,1.0,11,2,24.0,...,14.0,1849,1,11,0,1,0,1,0,1


In [3]:
fertil.describe()

,mnthborn,yearborn,age,electric,radio,tv,bicycle,educ,ceb,agefbrth,...,heduc,agesq,urban,urb_educ,spirit,protest,catholic,frsthalf,educ0,evermarr
count,4361.000000,4361.000000,4361.000000,4358.000000,4359.000000,4359.000000,4358.000000,4361.000000,4361.000000,3273.000000,...,1956.000000,4361.000000,4361.000000,4361.000000,4361.000000,4361.000000,4361.000000,4361.000000,4361.000000,4361.000000
mean,6.331346,60.433616,27.405182,0.140202,0.701766,0.092911,0.275815,5.855996,2.441642,19.011305,...,5.144683,826.459986,0.516625,3.469158,0.422151,0.227700,0.102499,0.540472,0.207751,0.476726
std,3.323333,8.682723,8.685233,0.347236,0.457535,0.290341,0.446975,3.927075,2.406861,3.092333,...,4.803028,526.923194,0.499781,4.294228,0.493959,0.419396,0.303339,0.498416,0.405744,0.499515
min,1.000000,38.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,...,0.000000,225.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,55.000000,20.000000,0.000000,0.000000,0.000000,0.000000,3.000000,1.000000,17.000000,...,0.000000,400.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.000000,62.000000,26.000000,0.000000,1.000000,0.000000,0.000000,7.000000,2.000000,19.000000,...,6.000000,676.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,9.000000,68.000000,33.000000,0.000000,1.000000,0.000000,1.000000,8.000000,4.000000,20.000000,...,8.000000,1089.000000,1.000000,7.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000
max,12.000000,73.000000,49.000000,1.000000,1.000000,1.000000,1.000000,20.000000,13.000000,38.000000,...,20.000000,2401.000000,1.000000,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
fertil.isnull().sum()

mnthborn       0
yearborn       0
age            0
electric       3
radio          2
tv             2
bicycle        3
educ           0
ceb            0
agefbrth    1088
children       0
knowmeth       7
usemeth       71
monthfm     2282
yearfm      2282
agefm       2282
idlnchld     120
heduc       2405
agesq          0
urban          0
urb_educ       0
spirit         0
protest        0
catholic       0
frsthalf       0
educ0          0
evermarr       0
dtype: int64

### Estimate the model: $ children = β0 + β1educ + β2age + β3age^2 + e $ by OLS, and interpret the estimates. In particular, holding age fixed, what is the estimated effect of another year of education on fertility? If 100 women receive another year of education, how many fewer children are they expected to have?

In [5]:
fertil_fit = smf.ols('children ~ educ + age + I(age**2)', fertil).fit()
fertil_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               children   R-squared:                       0.569
Model:                            OLS   Adj. R-squared:                  0.568
Method:                 Least Squares   F-statistic:                     1915.
Date:                Fri, 25 Nov 2022   Prob (F-statistic):               0.00
Time:                        11:35:44   Log-Likelihood:                -7835.6
No. Observations:                4361   AIC:                         1.568e+04
Df Residuals:                    4357   BIC:                         1.570e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -4.1383      0.241    -17.200      0.000      -4.610      -3.667
educ           -0.0906      0.006    -15.298      0.000      -0.102      -0.079
age             0.3324      0.017     20.088      0.000       0.300       0.365
I(age ** 2)    -0.0026      0.000     -9.651      0.000      -0.003      -0.002
==============================================================================
Omnibus:                      203.406   Durbin-Watson:                   1.868
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              715.951
Skew:                           0.017   Prob(JB):                    3.41e-156
Kurtosis:                       4.985   Cond. No.                     1.07e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.07e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [6]:
print("Another year of education decreases the number of children by", np.abs(fertil_fit.params[1]))

Another year of education decreases the number of children by 0.0905754619548432


In [7]:
print("If 100 women receive another year of education, they would expect to have", np.abs(fertil_fit.params[1]*100), "fewer children")

If 100 women receive another year of education, they would expect to have 9.05754619548432 fewer children


$age$ is tells that the older the woman gets, the higher the number of children she is going to have. Also, the square of $age$ suggests that the older the women get, the slower the rate which they have children.

### The variable $ frsthalf $ is a dummy variable equal to one if the woman was born during the first six months of the year. Assuming that frsthalf is uncorrelated with the error term from part (a), show that $ frsthalf $ is a reasonable IV candidate for $ educ $. (Hint: You need to do a regression.)

In [42]:
firsthalf_fit = smf.ols('educ ~ age + I(age**2) + frsthalf', fertil).fit()
firsthalf_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   educ   R-squared:                       0.108
Model:                            OLS   Adj. R-squared:                  0.107
Method:                 Least Squares   F-statistic:                     175.2
Date:                Fri, 25 Nov 2022   Prob (F-statistic):          3.01e-107
Time:                        18:05:00   Log-Likelihood:                -11905.
No. Observations:                4361   AIC:                         2.382e+04
Df Residuals:                    4357   BIC:                         2.384e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       9.6929      0.598     16.207      0.000       8.520      10.865
age            -0.1080      0.042     -2.568      0.010      -0.190      -0.026
I(age ** 2)    -0.0005      0.001     -0.730      0.466      -0.002       0.001
frsthalf       -0.8523      0.113     -7.554      0.000      -1.073      -0.631
==============================================================================
Omnibus:                       51.991   Durbin-Watson:                   1.352
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               56.555
Skew:                           0.233   Prob(JB):                     5.24e-13
Kurtosis:                       3.306   Cond. No.                     1.05e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.05e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Women born in the first half are likely to have 0.8523 years less of education. As the absolute value of the t-test for $frsthalf$ is 7.554, we can confirm that the variable is a strong IV.

### Estimate the model from part (a) by using $ frsthalf $ as an IV for $ educ $. Compare the estimated effect of education with the OLS estimate from part (a).

In [9]:
iv_fertil_fit = iv.IV2SLS.from_formula('children ~ 1 + age + I(age**2) [educ ~ frsthalf]', fertil).fit(cov_type = 'unadjusted')
iv_fertil_fit

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:               children   R-squared:                      0.5502
Estimator:                    IV-2SLS   Adj. R-squared:                 0.5499
No. Observations:                4361   F-statistic:                    5300.2
Date:                Fri, Nov 25 2022   P-value (F-stat)                0.0000
Time:                        11:40:06   Distribution:                  chi2(3)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept     -3.3878     0.5479    -6.1833     0.0000     -4.4617     -2.3139
I(age**2)     -0.0027     0.0003    -9.5589     0.0000     -0.0032     -0.0021
age            0.3236     0.0179     18.128     0.0000      0.2886      0.3586
educ          -0.1715     0.0532    -3.2264     0.0013     -0.2757     -0.0673
==============================================================================

Endogenous: educ
Instruments: frsthalf
Unadjusted Covariance (Homoskedastic)
Debiased: False
IVResults, id: 0x1d47b664340

In [10]:
print("The coefficient for educ in the regression in part (a) was:", fertil_fit.params[1])
print("The coefficient for educ in the 2 stages regression is", iv_fertil_fit.params[3])

The coefficient for educ in the regression in part (a) was: -0.0905754619548432
The coefficient for educ in the 2 stages regression is -0.1714989162709344


The effect of $educ$ on fertility in the 2 stages regression is bigger. Naturally, the SE also increased. 

### Add the binary variables $ electric$, $tv$, and $bicycle $ to the model and assume these are exogenous. Estimate the equation by OLS and 2SLS and compare the estimated coefficients on $educ$. Interpret the coefficient on $ tv $ and explain why television ownership has a negative effect on fertility.

In [13]:
tv_fertil_fit = smf.ols('children ~ educ + age + I(age**2) + electric + tv + bicycle', fertil).fit()
tv_fertil_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               children   R-squared:                       0.576
Model:                            OLS   Adj. R-squared:                  0.575
Method:                 Least Squares   F-statistic:                     984.9
Date:                Fri, 25 Nov 2022   Prob (F-statistic):               0.00
Time:                        11:46:35   Log-Likelihood:                -7789.3
No. Observations:                4356   AIC:                         1.559e+04
Df Residuals:                    4349   BIC:                         1.564e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -4.3898      0.240    -18.267      0.000      -4.861      -3.919
educ           -0.0767      0.006    -12.075      0.000      -0.089      -0.064
age             0.3402      0.016     20.692      0.000       0.308       0.372
I(age ** 2)    -0.0027      0.000    -10.010      0.000      -0.003      -0.002
electric       -0.3027      0.076     -3.974      0.000      -0.452      -0.153
tv             -0.2531      0.091     -2.768      0.006      -0.432      -0.074
bicycle         0.3179      0.049      6.440      0.000       0.221       0.415
==============================================================================
Omnibus:                      196.639   Durbin-Watson:                   1.891
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              675.502
Skew:                           0.009   Prob(JB):                    2.07e-147
Kurtosis:                       4.929   Cond. No.                     1.08e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.08e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

The coefficient for $tv$ implies that four families will have one child less if they own a television.

In [27]:
tviv_fertil_fit = iv.IV2SLS.from_formula('children ~ 1 + age + I(age**2) + electric + tv + bicycle [educ ~ frsthalf]', 
                                         fertil).fit(cov_type = 'unadjusted')
tviv_fertil_fit

C:\Users\leona\.anaconda\conda\lib\site-packages\linearmodels\iv\model.py:543: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(


IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:               children   R-squared:                      0.5577
Estimator:                    IV-2SLS   Adj. R-squared:                 0.5571
No. Observations:                4356   F-statistic:                    5539.2
Date:                Fri, Nov 25 2022   P-value (F-stat)                0.0000
Time:                        16:01:21   Distribution:                  chi2(6)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept     -3.5913     0.6446    -5.5717     0.0000     -4.8547     -2.3280
I(age**2)     -0.0027     0.0003    -9.8509     0.0000     -0.0033     -0.0022
age            0.3281     0.0190     17.231     0.0000      0.2908      0.3655
bicycle        0.3321     0.0515     6.4499     0.0000      0.2312      0.4330
electric      -0.1065     0.1658    -0.6424     0.5206     -0.4316      0.2185
tv            -0.0026     0.2091    -0.0122     0.9902     -0.4123      0.4072
educ          -0.1640     0.0655    -2.5045     0.0123     -0.2923     -0.0357
==============================================================================

Endogenous: educ
Instruments: frsthalf
Unadjusted Covariance (Homoskedastic)
Debiased: False
IVResults, id: 0x1d4003e4400

Having a television decreases of 0.0026 the number of children. The first possible interpretation is that people who own a television like to have sex less than their counterpart. On the other hand, having a television might also be a proxy variable for income, net worth, hours worked, etc. Indeed, I do not believe that being able to afford a tv in Botswana in 1988 was as common as today in the US. If this is true, a possible interpretation might be that richer people in 1988 Botswana made fewer children; it is reasonable as in rural countries people make kids to have more labor in their agricultural job. However, in particular, $tv$ is also usignificant (also $electric$). They might not be the best choice for this model as $children$ is a count variable.

The variable $educ$ becomes larger in the 2SLS regression. In the OLS, it is implying that an additional education year decreases the number of children of 0.0767, whereas, in the 2SLS regression, it is suggesting that children decrease by 0.164. In the second regression the SE also becomes larger, changing from 0.006 to 0.0655.

## PART 3

### Use smoke for this exercise.

In [3]:
smoke = woo.data('smoke')
smoke.head()

,educ,cigpric,white,age,income,cigs,restaurn,lincome,agesq,lcigpric
0,16.0,60.506001,1,46,20000,0,0,9.903487,2116,4.102743
1,16.0,57.882999,1,40,30000,0,0,10.308952,1600,4.058424
2,12.0,57.664001,1,58,30000,3,0,10.308952,3364,4.054633
3,13.5,57.882999,1,30,20000,0,0,9.903487,900,4.058424
4,10.0,58.320000,1,17,20000,0,0,9.903487,289,4.065945


In [12]:
smoke.describe()

,educ,cigpric,white,age,income,cigs,restaurn,lincome,agesq,lcigpric
count,807.000000,807.000000,807.000000,807.000000,807.000000,807.000000,807.000000,807.000000,807.000000,807.000000
mean,12.470880,60.300411,0.878563,41.237918,19304.832714,8.686493,0.246592,9.687316,1990.135068,4.096032
std,3.057161,4.738469,0.326837,17.027285,9142.958290,13.721516,0.431295,0.712696,1577.165644,0.082919
min,6.000000,44.004002,0.000000,17.000000,500.000000,0.000000,0.000000,6.214608,289.000000,3.784281
25%,10.000000,58.141499,1.000000,28.000000,12500.000000,0.000000,0.000000,9.433484,784.000000,4.062880
50%,12.000000,61.053001,1.000000,38.000000,20000.000000,0.000000,0.000000,9.903487,1444.000000,4.111742
75%,13.500000,63.179001,1.000000,54.000000,30000.000000,20.000000,0.000000,10.308952,2916.000000,4.145972
max,18.000000,70.128998,1.000000,88.000000,30000.000000,80.000000,1.000000,10.308952,7744.000000,4.250336


In [13]:
smoke.isnull().sum()

educ        0
cigpric     0
white       0
age         0
income      0
cigs        0
restaurn    0
lincome     0
agesq       0
lcigpric    0
dtype: int64

### Consider the following two structural equations. The first estimates the effects of smoking on annual income (perhaps through lost work days due to illness, or productivity effects) and the second is a demand equation reflecting the fact that cigarette consumption might be jointly determined with income: $log(income) = β0 + β1cigs + β2educ + β3age + β4age2 + e1$, $cigs = γ0 + γ1log(income) + γ2educ + γ3age + γ4age2 + γ5log(cigprice) + γ6restaurn + e2$, where cigprice is the price of a pack of cigarettes (in cents), and restaurn is a binary variable equal to unity if the person lives in a state with restaurant smoking restrictions. Assuming these are exogenous to the individual, what signs would you expect for $γ5$ and $6$?

I woould expect $\gamma5$ to be negative, as there is an negative relationship between the price and consumption of cigarettes. Moreover, $\gamma6$ should be negative as well, as non-smoking restaurants can be a reason to slightly justify a fewer consume of cigarettes. The might also be equal to 0.

### Under what assumption is the income equation from part (a) identified?

We need $\gamma5$ and $\gamma6$ statistically different from 0. This is because we need at least one of them to be significant, exogenous and not in the $log(income)$ equation.

### Estimate the income equation by OLS and also 2SLS. Discuss how the 2SLS estimate of β1 compares with the OLS estimate.

In [14]:
fit_inc = smf.ols('np.log(income) ~ cigs + educ + age + I(age**2)', smoke).fit()
fit_inc.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         np.log(income)   R-squared:                       0.165
Model:                            OLS   Adj. R-squared:                  0.161
Method:                 Least Squares   F-statistic:                     39.61
Date:                Mon, 28 Nov 2022   Prob (F-statistic):           2.68e-30
Time:                        14:56:33   Log-Likelihood:                -798.50
No. Observations:                 807   AIC:                             1607.
Df Residuals:                     802   BIC:                             1630.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       7.7954      0.170     45.741      0.000       7.461       8.130
cigs            0.0017      0.002      1.010      0.313      -0.002       0.005
educ            0.0604      0.008      7.642      0.000       0.045       0.076
age             0.0577      0.008      7.548      0.000       0.043       0.073
I(age ** 2)    -0.0006   8.34e-05     -7.563      0.000      -0.001      -0.000
==============================================================================
Omnibus:                      264.025   Durbin-Watson:                   1.908
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              960.742
Skew:                          -1.531   Prob(JB):                    2.38e-209
Kurtosis:                       7.381   Cond. No.                     1.88e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.88e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [15]:
fit_inc_iv = iv.IV2SLS.from_formula('np.log(income) ~ 1 + educ + age + I(age**2) [cigs ~ np.log(cigpric) + restaurn]', 
                                    smoke).fit(cov_type = 'unadjusted')
fit_inc_iv

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:         np.log(income)   R-squared:                     -0.5169
Estimator:                    IV-2SLS   Adj. R-squared:                -0.5245
No. Observations:                 807   F-statistic:                    89.802
Date:                Mon, Nov 28 2022   P-value (F-stat)                0.0000
Time:                        14:56:33   Distribution:                  chi2(4)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      7.7809     0.2292     33.955     0.0000      7.3318      8.2300
I(age**2)     -0.0011     0.0003    -3.8424     0.0001     -0.0016     -0.0005
age            0.0938     0.0238     3.9454     0.0001      0.0472      0.1404
educ           0.0397     0.0162     2.4444     0.0145      0.0079      0.0715
cigs          -0.0421     0.0261    -1.6117     0.1070     -0.0934      0.0091
==============================================================================

Endogenous: cigs
Instruments: np.log(cigpric), restaurn
Unadjusted Covariance (Homoskedastic)
Debiased: False
IVResults, id: 0x1e96dd0bd00

In [16]:
print("The coefficient for cigs in the OLS regression was:", fit_inc.params[1])
print("The coefficient for cigs in the 2SLS regression is", fit_inc_iv.params[4])

The coefficient for cigs in the OLS regression was: 0.0017305743972014856
The coefficient for cigs in the 2SLS regression is -0.04212574507891986


The coefficient for $cigs$ decreased and became negative in the 2SLS regression. However, it gained significance; now it is almost significant at for the 10% level. The interpretation of the OLS coefficient assumed that for each additional cigarette, the income increases by 0.17%, which is economically ambiguos. The 2SLS coefficient, on the other hand, implies that for each additional cigarette, income is reduced by about 4.21%

### Do you think that cigarette prices and restaurant smoking restrictions are exogenous in the income equation?

It is hard to say whether $cigpric$ and $restaurn$ are exogenous or endogenous. It can be argued that smoking restriction might be exogenous as it mainly depends on each state laws and not on income in general, yet aslo income might depend on each state as the socio-economic conditions are different. Cigarette prices might be exogenous in the income equation as they are external factors in determining income.

In [49]:
fit_inc_iv.sargan

Sargan's test of overidentification
H0: The model is not overidentified.
Statistic: 1.3559
P-value: 0.2443
Distributed: chi2(1)
WaldTestStatistic, id: 0x1e96cce8b50

The null hypothesis is accepted, hence the model might be correctly identified; there is no extra instrument to test for endogeneity and the IVs are jointly exogenous.